In [17]:
# code to generate the C function for endmembers (pure species)

# last update 25.01.22, NR

# Load needed libraries
# import nlopt
import numpy as np
import math as math
import cmath
import random

directory = 'database_Igneous_H2018'
database  = 'tc-ds634.txt'
path      = directory+'/'+database
out_name  = 'MAGEMin_'+database

In [18]:
def func_print(data):
    txt ='{'
    for i,val in enumerate(data):
        if i == len(data)-1:
            txt += str(val)
        else:
            txt += str(val)+', ' 
    
    txt +='}'    
    return(txt)

In [19]:
#first clean up the end-member file, remove first and last lines of the file

gem = open(path,'r')

#declare endmember structure, name + thermodynamic data
em = {}
new_content = [[] for x in range(4)] #create 2d array with 4 lines and unspecified column number

#read thermocalc endmember file and fill endmember structure
c=1;
for line in gem:
    words = line.split()
    count = len(words)

    if c == 1: #attribute endmember name to the dictionary
        name0 = words[0]
        content = np.asarray(words[1:count])   
        new_content[0] = [float(i) for i in content]
    elif c > 1 and c < 5: #attribute thermodynamic data to the endmember dictionary entry
        content = np.asarray(words[0:count]) 
        new_content[c-1] = [float(i) for i in content]  

    em[name0] = new_content[0:4]

    c+=1
    if c == 5:
        c = 1
        
apo = [3.0,5.0,2.0,2.0,2.0,3.0,3.0,3.0,1.0,5.0,3.0]; #number of atoms per oxide

# generate the C function
sp = '    '
op = ''
ip = ''
ip += 'char em_list['+str(len(em)-1)+'][5] = {'
for ix,i in enumerate(em):
    if ix <(len(em)-1):
        ip += '"' + str(i)+'", '
    else:
        ip += '"' + str(i)+'"'
ip += '};'

op += 'struct em_db arr_em_db['+str(len(em))+'] = {\n'

for ix,i in enumerate(em):
    op += sp +  '{\n'
  
    op += sp + sp + '"' + str(i)+'",\n'
    for jx,j in enumerate(em[i]):
        if jx == 0:
            #Chemical composition
            el_comp = np.zeros(19);
            chem_comp = np.zeros(11);

            y  = 1;
            l = len(em[i][0])-1;

            while y < l:
                el_comp[int(em[i][0][y])-1] = em[i][0][y+1];
                y = y + 2;

            #Normalization of endmember composition
            chem_comp = [el_comp[0], el_comp[2]/2, el_comp[6], el_comp[4], el_comp[3], el_comp[8]/2,el_comp[7]/2, el_comp[1],el_comp[9], el_comp[18]/2,el_comp[10]/2];

            if 2*chem_comp[0] + 3*chem_comp[1] + chem_comp[2] + chem_comp[3] + chem_comp[4] + chem_comp[5] +  chem_comp[6] + 2*chem_comp[7] + 3*chem_comp[9]+ chem_comp[10] != el_comp[8]:
                chem_comp[8] =  chem_comp[8] - (2*chem_comp[0] + 3*chem_comp[1] + chem_comp[2] + chem_comp[3] + chem_comp[4] + chem_comp[5] +  chem_comp[6] + 2*chem_comp[7] + 3*chem_comp[9]+ chem_comp[10]);
            
            apf = sum(np.array(chem_comp)*np.array(apo))
            chem_comp.append(apf)
            
            op += sp+ sp +func_print(chem_comp)+',\n'
            
        elif jx < 3 and jx > 0:
            if jx == 1:
                op += sp+ sp +func_print(j)+',\n'
            if jx == 2:
                op += sp+ sp +func_print(j)+',\n' 

        else:
            third_line = [0.0]*11
            for k,val in enumerate(j):
                third_line[k] = val
            op += sp+ sp +func_print(third_line)+'\n'
            
    if ix < len(em)-1:
        op += sp + '},\n'
    else:
        op += sp + '}\n'
op += '};\n\n'


In [20]:
print(op)
text_file   = open(out_name, "w")
n           = text_file.write(op)

struct em_db arr_em_db[291] = {
    {
        "fo",
        {1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0},
        {-2172.45, 0.0951, 4.366},
        {0.2333, 1.494e-06, -603.8, -1.8697},
        {2.85e-05, 1285.0, 3.84, -0.003, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0}
    },
    {
        "fa",
        {1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0},
        {-1477.52, 0.151, 4.631},
        {0.2011, 1.733e-05, -1960.6, -0.9009},
        {2.82e-05, 1256.0, 4.68, -0.0037, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0}
    },
    {
        "teph",
        {1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0},
        {-1733.86, 0.1559, 4.899},
        {0.2196, 0.0, -1292.7, -1.3083},
        {2.86e-05, 1256.0, 4.68, -0.0037, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0}
    },
    {
        "lrn",
        {1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0},
        {-2306.86, 0.1276, 5.16},
        {0.2475, -3.206e-06, 0.0, -2.0519},
        {2.9e-05, 985.0, 4.07, -0.0041